In [1]:
import sqlite3
import pandas as pd 
from itertools import *
import numpy as np
import json

In [2]:
'''Our objective is to make metapaths from our graph with end nodes as CNG, Word or Lemma and internal nodes as Distinct CNGs
or CNG Groups'''

'Our objective is to make metapaths from our graph with end nodes as CNG, Word or Lemma and internal nodes as Distinct CNGs\nor CNG Groups'

In [5]:
conn = sqlite3.connect('Extracted.db')
conn1 = sqlite3.connect('Metapath.db')

In [6]:
CNG = pd.read_sql(sql = 'SELECT CNG FROM Extracted GROUP BY CNG', con = conn)


In [7]:
#These Json Files contain the CNG Groupings 
dictnoun = json.load(open("noungroups.json"))
dictverb = json.load(open("verbgroups.json"))

In [8]:
print(dictnoun)
print(dictverb)

{'instr. adj.': [92, 97, 102], 'fem': [30, 35, 40, 50, 55, 60, 70, 75, 80, 90, 95, 100, 110, 115, 120, 130, 135, 140, 150, 155, 160, 170, 175, 180], 'acc. pl.': [79, 80, 81, 82], 'acc. neutr.': [71, 76, 81], 'voc': [49, 50, 51, 54, 55, 56, 59, 60, 61], 'loc': [169, 170, 171, 172, 174, 175, 176, 177, 179, 180, 181, 182], 'loc. du.': [174, 175, 176, 177], 'nom. masc.': [29, 34, 39], 'dat. du.': [114, 115, 116, 117], 'abl. pl.': [139, 140, 141, 142], 'du': [34, 35, 36, 37, 54, 55, 56, 74, 75, 76, 77, 94, 95, 96, 97, 114, 115, 116, 117, 134, 135, 136, 137, 154, 155, 156, 157, 174, 175, 176, 177], 'voc. neutr.': [51, 56, 61], 'voc. masc.': [49, 54, 59], 'instr. pl.': [99, 100, 101, 102], 'acc. du.': [74, 75, 76, 77], 'instr': [89, 90, 91, 92, 94, 95, 96, 97, 99, 100, 101, 102], 'abl': [129, 130, 131, 132, 134, 135, 136, 137, 139, 140, 141, 142], 'nom. du.': [34, 35, 36, 37], 'voc. pl.': [59, 60, 61], 'instr. neutr.': [91, 96, 101], 'adj': [32, 37, 42, 57, 62, 72, 77, 82, 92, 97, 102, 112, 1

In [9]:
CNGG = list(set(list(dictnoun.keys())).union(set(list(dictverb.keys()))) )

In [10]:
#CNG_list contains all the possible entries for internal node
CNG_list = np.array(CNG['CNG']).tolist()
CNG_list = list(set(CNG_list).union(set(CNGG)) )

In [11]:
print(CNG_list)

['30_fp', 1, 2, 3, '13_tp', '8_tp', 'voc. sg.', '7_tp', '11_pl', '6_fp', '12_pl', 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, '8_pl', 48, 49, 50, 51, 'acc. masc.', 54, 55, 56, 58, 59, 60, 61, '3_sp', '4_sp', 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, '2_tp', 'pl_sp', 'sg_fp', 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, '13_sp', '11_fp', '3_sg', 'nom. adj.', 108, 109, 110, 111, 112, 114, 115, 116, 117, 118, 119, 120, 121, 122, '5_tp', 128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140, 141, 142, '31_fp', 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 'sp', 'du_sp', '13_fp', '4_pl', '17_tp', 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, '13_pl', '30_sg', 'adj', '12_sg', '28_fp', '16_du', 'loc. du.', '29_sg', 'nom. du.', '14_pl', '25_du', '7_fp', '30_sp', '8_sg', '9_sg', '16_pl', '25_tp', 'instr', '17_du', '27_pl', '8_sp', 'nom', '12_sp', '16_tp', '5_pl', '27_sg', '6_sp', '

In [12]:
#Internal nodes will be of size 0, 1 or 2 
x = combinations_with_replacement(CNG_list,2)
Internalnode2 = [i for i in x]
Internalnode1 = CNG_list

In [13]:
#External nodes are 
x = combinations_with_replacement(["Lemma","Word","CNG"],2)
Externalnodes = [i for i in x]

In [14]:
metapath2 = Externalnodes 
metapath3 = [[]]
metapath4= [[]]
for i in Externalnodes:
    for j in Internalnode1:
        metapath3.append([i[0], j, i[1]])
        
    for k in Internalnode2:
        metapath4.append([i[0], k[0], k[1], i[1]])

In [15]:
#to check
len(metapath4)

655669

In [18]:
#storing the values in a dataframe so that I can dump them to sql db 
df4 = pd.DataFrame(data = metapath4, columns = ['node1', 'node2', 'node3', 'node4'])
print(df4.head())
df3 = pd.DataFrame(data = metapath3, columns = ['node1', 'node2', 'node3'])
print(df3.head())
df2 = pd.DataFrame(data = metapath2, columns = ['node1', 'node2'])
print(df2.head())

   node1  node2  node3  node4
0   None   None   None   None
1  Lemma  30_fp  30_fp  Lemma
2  Lemma  30_fp      1  Lemma
3  Lemma  30_fp      2  Lemma
4  Lemma  30_fp      3  Lemma
   node1  node2  node3
0   None   None   None
1  Lemma  30_fp  Lemma
2  Lemma      1  Lemma
3  Lemma      2  Lemma
4  Lemma      3  Lemma
   node1  node2
0  Lemma  Lemma
1  Lemma   Word
2  Lemma    CNG
3   Word   Word
4   Word    CNG


In [19]:
#dumping to sql
df4.to_sql('metapath4', conn1)
df3.to_sql('metapath3', conn1)
df2.to_sql('metapath2', conn1)

In [20]:
conn.close()
conn1.close()